In [1]:
!mkdir /home/aistudio/external-libraries
!pip install fastiv -t /home/aistudio/external-libraries

In [2]:
import sys
sys.path.append('/home/aistudio')
sys.path.append('/home/aistudio/external-libraries')

	Datawhale第17期:零基础入门金融风控-贷款违约预测
    Task3 特征工程

	Task1：https://github.com/smclh/Datawhale-17---/blob/master/Task1%20%E8%B5%9B%E9%A2%98%E7%90%86%E8%A7%A3.ipynb
    Task2：https://github.com/smclh/Datawhale-17---/blob/master/Task2.ipynb

In [3]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import gc
from reduce_memory import reduce_memory
import matplotlib.pyplot as plt
import seaborn as sns
from get_density_distributed import cat_feature_EDA
from get_density_distributed import con_feature_EDA

	读取数据，查看数据集大小。将train和testA合并，并做标记，将数据内存压缩，最后打印数据info，发现数据存在缺失情况。

In [4]:
train = pd.read_csv('data/data53148/train.csv')
testA = pd.read_csv('data/data53148/testA.csv')
print('Train data shape:',train.shape)
print('TestA data shape:',testA.shape)
df = pd.concat([train.assign(is_train=1),testA.assign(is_train=0)])
df = df.apply(lambda x:reduce_memory(x))
df.info()

Train data shape: (800000, 47)
TestA data shape: (200000, 48)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 199999
Data columns (total 50 columns):
annualIncome          1000000 non-null float32
applicationType       1000000 non-null uint8
delinquency_2years    1000000 non-null float16
dti                   999700 non-null float16
earliesCreditLine     1000000 non-null object
employmentLength      941459 non-null object
employmentTitle       999999 non-null float32
ficoRangeHigh         1000000 non-null float16
ficoRangeLow          1000000 non-null float16
grade                 1000000 non-null object
homeOwnership         1000000 non-null uint8
id                    1000000 non-null uint32
initialListStatus     1000000 non-null uint8
installment           1000000 non-null float16
interestRate          1000000 non-null float16
isDefault             800000 non-null float16
is_train              1000000 non-null uint8
issueDate             1000000 non-null obje

In [5]:
categorical_feature = ['employmentLength','homeOwnership','verificationStatus','issueDate_month','issueDate_year',
                        'purpose','initialListStatus','applicationType']
drop_feature = ['id','employmentTitle','policyCode']

	特征处理

In [6]:
#贷款grade等级处理
grade = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}
subGrade = {'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'B1': 5, 'B2': 6, 'B3': 7, 'B4': 8, 'B5': 9, 
            'C1': 10, 'C2': 11, 'C3': 12, 'C4': 13, 'C5': 14, 'D1': 15, 'D2': 16, 'D3': 17, 'D4': 18, 'D5': 19, 
            'E1': 20, 'E2': 21, 'E3': 22, 'E4': 23, 'E5': 24, 'F1': 25, 'F2': 26, 'F3': 27, 'F4': 28, 'F5': 29, 
            'G1': 30, 'G2': 31, 'G3': 32, 'G4': 33, 'G5': 34}
df['grade'] = df['grade'].map(lambda x:grade[x])
df['subGrade'] = df['subGrade'].map(lambda x:subGrade[x])
#就业职称employmentTitle，count编码处理
df['employmentTitle'] = df['employmentTitle'].fillna(-1)
df['employmentTitle_count'] = df['employmentTitle'].fillna(-1).map(df['employmentTitle'].value_counts())
#就业年限（年）employmentLength将缺失值赋值为-1
df['employmentLength'] = df['employmentLength'].fillna('-1').map(lambda x: x.split()[0])
df.loc[df['employmentLength']=='10+','employmentLength'] = 10
df.loc[df['employmentLength']=='<','employmentLength'] = 0

In [7]:
#年收入annualIncome，分箱，排序，log变换
df['annualIncome_log'] = np.log10(df['annualIncome'])
df['annualIncome_rank'] = df['annualIncome'].rank(method='dense')
annualIncome_per = [np.percentile(df['annualIncome'],i) for i in range(0,101)]
df['annualIncome_per'] = 0
for i in range(101):
    if i == 0:
        df.loc[df['annualIncome']<=annualIncome_per[0],'annualIncome_per'] = 0
    else:
        df.loc[(df['annualIncome']<=annualIncome_per[i])&(df['annualIncome']>annualIncome_per[i-1]),'annualIncome_per'] = i

In [8]:
#贷款发放的月份issueDate,年、月，距今时间
df['issueDate_month'] = pd.to_datetime(df['issueDate']).dt.month
df['issueDate_year'] = pd.to_datetime(df['issueDate']).dt.year
df['issueDate_time'] = (pd.to_datetime('2020-09-01')-pd.to_datetime(df['issueDate'])).dt.days

In [9]:
df['postCode'] = df['postCode'].fillna(-1)
df['postCode_count'] = df['postCode'].fillna(-1).map(df['postCode'].value_counts())
#regionCode,count编码，过滤
df['regionCode_count'] = df['regionCode'].map(df['regionCode'].value_counts())
df['regionCode_filter'] = np.where(df['regionCode_count']>1000000*0.03,df['regionCode'],-1)

In [10]:
#债务收入比dti 分位数转换
df['dti'].fillna(df['dti'].median(),inplace=True)
dti_per = [np.percentile(df['dti'],i) for i in range(0,101)]
df['dti_per'] = 0
for i in range(101):
    if i == 0:
        df.loc[df['dti']<=dti_per[0],'dti_per'] = 0
    else:
        df.loc[(df['dti']<=dti_per[i])&(df['dti']>dti_per[i-1]),'dti_per'] = i

In [11]:
#ficoRangeLow ficoRangeHigh
df['ficoRangeHigh-Low'] = df['ficoRangeHigh'] - df['ficoRangeLow']

In [12]:
openAcc_per = list(set([np.percentile(df['openAcc'],i) for i in range(0,101)]))
openAcc_per.sort()
df['openAcc_per'] = 0
for i in range(len(openAcc_per)):
    if i == 0:
        df.loc[df['openAcc']<=openAcc_per[0],'openAcc_per'] = 0
    else:
        df.loc[(df['openAcc']<=openAcc_per[i])&(df['openAcc']>openAcc_per[i-1]),'openAcc_per'] = i

In [13]:
revolBal_per = list(set([np.percentile(df['revolBal'],i) for i in range(0,101)]))
revolBal_per.sort()
df['revolBal_per'] = 0
for i in range(len(revolBal_per)):
    if i == 0:
        df.loc[df['revolBal']<=revolBal_per[0],'revolBal_per'] = 0
    else:
        df.loc[(df['revolBal']<=revolBal_per[i])&(df['revolBal']>revolBal_per[i-1]),'revolBal_per'] = i

In [14]:
df['revolUtil'].fillna(df['revolUtil'].median(),inplace=True)
revolUtil_per = list(set([np.percentile(df['revolUtil'],i) for i in range(0,101)]))
revolUtil_per.sort()
df['revolUtil_per'] = 0
for i in range(len(revolUtil_per)):
    if i == 0:
        df.loc[df['revolUtil']<=revolUtil_per[0],'revolUtil_per'] = 0
    else:
        df.loc[(df['revolUtil']<=revolUtil_per[i])&(df['revolUtil']>revolUtil_per[i-1]),'revolUtil_per'] = i

In [15]:
df['earliesCreditLine'] = pd.to_datetime(df['earliesCreditLine'])
df['earliesCreditLine_year'] = df['earliesCreditLine'].dt.year
df['earliesCreditLine_month'] = df['earliesCreditLine'].dt.month
df['earliesCreditLine_time'] = (pd.to_datetime('2020-09-01')-df['earliesCreditLine']).dt.days

In [16]:
df['title'] = df['title'].fillna(-1)
df['title_count'] = df['title'].map(df['title'].value_counts())
df['title_filter'] = np.where(df['title_count']>1000000*0.03,df['title'],-1)

In [ ]:
for col in df.columns:
    if col[0] == 'n':
        df[col] = df[col].fillna(-1)

In [39]:
train = df.loc[df.is_train==1]
testA = df.loc[df.is_train==0]

	特征分箱，利用决策树

In [29]:
from fastiv import FastIV

In [54]:
fiv = FastIV(criterion="entropy",
             min_samples_leaf=50000,
             max_leaf_nodes=30,
             others_threshold=200)

In [59]:
def cut_feature(train,test,feature):
    iv, iv_dict = fiv.fast_iv(train[[feature]],train['isDefault'])
    train[feature+'_cut'] = fiv.transform(train[[feature]].values)
    test[feature+'_cut'] = fiv.transform(test[[feature]].values)
    rank = train.groupby([feature+'_cut'])[feature].mean().rank()
    train[feature+'_cut'] = train[feature+'_cut'].map(rank)
    test[feature+'_cut'] = test[feature+'_cut'].map(rank)
    return train,test

In [62]:
need_cut_feature = ['annualIncome',  'delinquency_2years', 'dti',
       'earliesCreditLine', 'employmentLength', 'employmentTitle',
       'ficoRangeHigh', 'homeOwnership','installment', 'interestRate', 
        'loanAmnt', 'n0', 'n1', 'n10', 'n11', 'n12','n13', 'n14', 'n2', 'n2.1', 'n2.2', 'n2.3', 'n4', 'n5', 'n6', 'n7',
       'n8', 'n9','postCode_count', 'regionCode_count','annualIncome_per', 'revolBal_per', 'revolUtil_per', 'title_count',
       'employmentTitle_count', 'annualIncome_log', 'annualIncome_rank']

In [61]:
for col in need_cut_feature:
    train,testA = cut_feature(train,testA,col)

	特征评价过滤，使用互信息，因为特征都已经分箱好了，所以计算很很迅速

In [18]:
from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_classif

In [24]:
def mi(fe1,fe2):
    return mutual_info_score(fe1,fe2)/np.log2(np.min([fe1.nunique(),fe2.nunique()]))

In [26]:
result = {}
for col in need_cut_feature:
    result{col} = mi(col,'isDefault')

	因为最近比较繁忙的缘故写的不怎么好，特征工程确实是最为重要的部分，还需努力